<div style="text-align: right; font-size: 1.2em;">
    <img src="logo-uec.png" alt="Logo Universidad" style="width: 150px; float: right; margin-left: 20px;"/>
    <b>Maestría en Inteligencia de Negocios</b>
    <br>
    <b>Curso: Tópicos de Machine Learning y Redes Neuronales</b>
</div>

# Sesión 2: La Revolución del Boosting 🚀

## Introducción

En nuestra sesión anterior, exploramos el poder de los **métodos de ensamble** con un enfoque en **Bagging** y su implementación más famosa: **Random Forest**. Vimos cómo la "sabiduría de la multitud", al promediar las predicciones de muchos árboles (entrenados en paralelo en diferentes subconjuntos de datos), puede reducir drásticamente la varianza y mejorar la robustez del modelo.

Hoy, vamos a explorar una filosofía de ensamble diferente pero increíblemente poderosa: **Boosting**.

Si Random Forest es una democracia (donde cada árbol vota), Boosting es una jerarquía de expertos donde cada nuevo experto aprende de los errores del anterior.

## 1. Boosting: Aprendiendo de los Errores

La idea fundamental del Boosting es entrenar modelos **secuencialmente**. Cada nuevo modelo se enfoca en corregir los errores cometidos por la secuencia de modelos anteriores.

A diferencia de Bagging (Random Forest) donde los árboles se construyen de forma independiente y en paralelo, en Boosting el proceso es **secuencial y adaptativo**.

--- 

### Una Analogía Simple

Imagina que eres un estudiante preparándote para un examen difícil:

1.  **Intento 1 (Modelo 1):** Haces un examen de práctica. Obtienes un 70%. Algunas preguntas te parecieron fáciles (las acertaste) y otras muy difíciles (las fallaste).
2.  **Aprendizaje (Error):** Recibes tu calificación. Ahora sabes exactamente cuáles son tus puntos débiles (las preguntas que fallaste).
3.  **Intento 2 (Modelo 2):** En lugar de volver a estudiar *todo* el material, te enfocas intensamente en los temas de las preguntas que fallaste. Haces un nuevo examen de práctica sobre *esos* temas.
4.  **Conocimiento Final (Ensamble):** Tu conocimiento final no es solo tu último examen, sino una combinación ponderada de tu conocimiento del primer intento y tu conocimiento enfocado del segundo intento.

--- 

### Contraste: Bagging vs. Boosting

| Característica | Bagging (Ej. Random Forest) | Boosting (Ej. Gradient Boosting) |
| :--- | :--- | :--- |
| **Proceso** | Paralelo (Árboles independientes) | Secuencial (Un árbol aprende del anterior) |
| **Objetivo Principal** | Reducir Varianza | Reducir Sesgo (y luego varianza) |
| **Manejo de Errores** | Promedia los errores | Se enfoca en los errores y los corrige |
| **Ponderación** | Todos los modelos votan por igual | Los modelos se ponderan (algunos tienen más peso) |
| **Riesgo** | Generalmente robusto al overfitting | Puede sobreajustar si se usan demasiados árboles |


## 2. El Algoritmo de Boosting y la Tasa de Aprendizaje

El algoritmo que define el "Gradient Boosting" moderno se popularizó en el artículo "Greedy Function Approximation: A Gradient Boosting Machine" (Friedman, 2001) y es el que se muestra en la imagen que compartiste (Algoritmo 8.2 de Hastie & Tibshirani).

Vamos a desglosarlo:

### El Algoritmo Base (Visto en Hastie & Tibshirani)

1.  **Inicializar:** Se fija la predicción inicial `f(x) = 0`. El "error" o **residuo** inicial (`r_i`) es simplemente el valor real que queremos predecir: `r_i = y_i`.

2.  **Iterar** para `b = 1` hasta `B` (donde `B` es el número total de árboles que queremos entrenar):
    
    a.  Entrenar un árbol de regresión `h^b(x)` para que prediga los **residuos actuales** `r_i` (usando `X` como variables predictoras y `r` como variable objetivo).
    
    b.  Actualizar la predicción total `f(x)` añadiendo una *versión "encogida"* del nuevo árbol:
        `f(x) <-- f(x) + λ * h^b(x)`
        
    c.  Actualizar los residuos para la *siguiente* iteración (el siguiente árbol):
        `r_i <-- r_i - λ * h^b(x)`

3.  **Resultado Final:** El modelo final `F(x)` es la suma de todas las contribuciones de los árboles: `F(x) = Σ [λ * h^b(x)]`

--- 

### ¿Qué es la Tasa de Aprendizaje (`λ`)?

El parámetro `λ` (lambda) se conoce como la **tasa de aprendizaje** (*learning rate*) o *encogimiento* (*shrinkage*). Es uno de los hiperparámetros más importantes del modelo.

* **¿Qué hace?** Controla *cuánto* del aprendizaje del nuevo árbol (`h^b(x)`) añadimos a la predicción total en cada paso.
* **¿Por qué?** Es la principal herramienta para **prevenir el overfitting**. Si `λ = 1`, cada nuevo árbol intenta corregir *todo* el error restante, lo cual es muy agresivo y lleva a que el modelo memorice el ruido de los datos de entrenamiento.
* **La Estrategia:** Es mejor "dar pasos pequeños y seguros". Al usar un `λ` pequeño (ej. 0.1, 0.05, o 0.01), cada árbol solo corrige una pequeña fracción del error. Esto hace que el aprendizaje sea más lento, pero mucho más robusto y generalizable.

**La compensación (Trade-off):**
* Un `λ` **pequeño** requiere un `B` (número de árboles) **grande** para alcanzar un buen rendimiento.
* Un `λ` **grande** aprende rápido (requiere menos árboles), pero es muy probable que sobreajuste.

--- 

### Ejemplo Intuitivo (con Tasa de Aprendizaje)

Volvamos a nuestro ejemplo de predecir precios de casas, pero ahora con `λ = 0.1`.

**Paso 1: El Modelo Base (Árbol 1)**
Predicción promedio: **$300k**.
Residuo (Casa A): $350k (Real) - $300k (Pred) = **+$50k**.

**Paso 2: El Segundo Modelo (Árbol 2)**
El Árbol 2 se entrena con los residuos. Supongamos que para la Casa A, predice un residuo de **+$40k**.

**Paso 3: Combinar las Predicciones (con `λ = 0.1`)**

`Predicción Actualizada = Predicción Anterior + λ * Predicción Árbol 2`
`Predicción Actualizada = $300k + 0.1 * $40k`
`Predicción Actualizada = $300k + $4k = $304k`

Nuestra nueva predicción es **$304k**. Nótese que no saltamos directamente a $340k. Nos movemos solo un 10% del camino.

**Paso 4: Repetir (Iterar)**

Ahora, calculamos un *nuevo* residuo para el siguiente árbol:

`Nuevo Residuo = Valor Real - Predicción Actualizada`
`Nuevo Residuo = $350k - $304k = +$46k`

El **Árbol 3** se entrenará para predecir este nuevo residuo de **+$46k**. El proceso se repite, refinando lentamente la predicción.

## 3. ¿Y AdaBoost? ¿Cuál es la Diferencia?

Es común confundir AdaBoost (Adaptive Boosting) con Gradient Boosting, pero funcionan de manera fundamentalmente diferente.

**AdaBoost (Adaptive Boosting):**

1.  **Ponderación de Muestras:** AdaBoost comienza dando a todas las muestras (filas) de datos el mismo peso o importancia.
2.  **Entrenar Modelo Simple:** Entrena un modelo simple (usualmente un "árbol tocón" o *stump*, un árbol con una sola división).
3.  **Identificar Errores:** Mira qué muestras clasificó *mal*.
4.  **Adaptar (Adaptive):** *Aumenta el peso* (la importancia) de esas muestras mal clasificadas. Las muestras que se clasificaron bien ven su peso reducido.
5.  **Entrenar Siguiente Modelo:** El segundo modelo se entrena, pero ahora tiene una *penalización extra* por fallar en las muestras que el primer modelo ya falló (porque ahora "valen más").

**Diferencia Clave:**

| | **Gradient Boosting (GBM)** | **AdaBoost** |
| :--- | :--- | :--- |
| **En qué se enfoca** | En el **error de predicción (Residuo)**. | En las **muestras (filas)** mal clasificadas. |
| **Cómo aprende** | El siguiente modelo predice los *residuos* del modelo anterior. | El siguiente modelo da más importancia a las *muestras* que el modelo anterior falló. |
| **Variable Objetivo** | La "y" cambia en cada paso (pasa a ser el residuo). | La "y" siempre es la misma, pero los *pesos de las muestras* cambian. |


## 4. ¿Dónde entra el "Gradiente" en Gradient Boosting?

En un modelo de regresión lineal (OLS), ¿cómo encontramos los mejores coeficientes? Minimizando una **función de pérdida**, la Suma de Errores al Cuadrado (SEC): `Σ(y_real - y_pred)²`.

¿Y cómo minimizamos cualquier función en cálculo? Encontrando dónde su **gradiente** (la derivada) es cero. En Machine Learning, usamos un algoritmo llamado **Gradiente Descendente**: calculamos el gradiente y nos movemos en la dirección opuesta (el *gradiente negativo*) para reducir el error. 

### ¡Los "Residuos" son un Gradiente!

La genialidad de Friedman (el creador de GBM) fue darse cuenta de esto:

> **Para la función de pérdida de Error Cuadrático (SEC), el *gradiente negativo* es exactamente el residuo (`y_real - y_pred`).**

El algoritmo que vimos en la Sección 2 (entrenar árboles sobre los residuos) es, de hecho, una forma de **Gradiente Descendente** en el "espacio de las funciones". Cada nuevo árbol `h(x)` es un pequeño paso en la dirección que más reduce el error.

### ¿Por qué llamarlo "Gradient" Boosting?

Porque esto **generaliza** el algoritmo. Ya no estamos limitados a usar residuos (que solo funcionan para el Error Cuadrático).

Podemos usar **cualquier función de pérdida que sea derivable**:

* **Clasificación:** Usamos *Log-Loss* (Pérdida Logarítmica).
* **Regresión Robusta:** Usamos *MAE* (Error Absoluto Medio) o *Huber Loss*.

En cada paso, el nuevo árbol no se entrena para predecir el residuo, sino para predecir el **gradiente negativo de la función de pérdida** de cada muestra. Esto es lo que hace que GBM sea una navaja suiza para casi cualquier problema de ML supervisado.

## 5. Los Campeones de la Industria: XGBoost y LightGBM

Si GBM es tan bueno, ¿por qué todos usan **XGBoost** y **LightGBM**? Porque toman esta idea del gradiente y la optimizan a un nivel extremo (de ahí el nombre "eXtreme Gradient Boosting").

--- 

### XGBoost (eXtreme Gradient Boosting)

**Mejoras Clave:**

1.  **La Mejora del Gradiente (La conexión):** XGBoost lleva la idea del gradiente un paso más allá. 
    * GBM clásico usa solo el **gradiente** (la "primera derivada" de la función de pérdida). 
    * XGBoost usa *ambos*, el **gradiente** (1ª derivada) y el **Hessiano** (la "segunda derivada").
    * **Intuición:** Si el gradiente (1ª derivada) te dice *en qué dirección* moverte para reducir el error (la *pendiente*), el Hessiano (2ª derivada) te dice *cuánto* debes moverte para encontrar el mínimo (la *curvatura*). Usar ambos es mucho más rápido y preciso, como un método de Newton-Raphson.

2.  **Regularización (L1 y L2):** Esta es su arma secreta. Al igual que en la Regresión Lineal (Ridge y Lasso), XGBoost añade una penalización a la complejidad del modelo (profundidad, número de hojas), previniendo el sobreajuste por diseño.

3.  **Optimización de Velocidad (Paralelismo):** Paraleliza la *construcción de cada árbol* (evaluando cortes en paralelo).

4.  **Manejo de Valores Nulos (Missing Values):** Aprende la mejor "ruta" para los valores nulos durante el entrenamiento.

--- 

### LightGBM (Light Gradient Boosting Machine)

Desarrollado por Microsoft, es (generalmente) aún más rápido que XGBoost.

**Mejoras Clave:**

1.  **Misma base de Gradiente/Hessiano:** Usa la misma optimización de 1ª y 2ª derivada que XGBoost.

2.  **Velocidad Extrema (Crecimiento *Leaf-wise*):**
    * *GBM/XGBoost (Level-wise):* Construyen el árbol nivel por nivel, de forma simétrica.
    * *LightGBM (Leaf-wise):* Crece el árbol de forma asimétrica. Identifica la hoja que *más reducirá la pérdida* y la expande. Es más rápido y preciso, pero puede sobreajustar en datasets pequeños.

3.  **Manejo Eficiente de Variables Categóricas:**
    * Puede manejarlas de forma nativa sin necesidad de One-Hot Encoding, ahorrando muchísima memoria y tiempo.

### Conclusión de la Teoría

Hemos pasado del concepto de "aprender de los errores" (Boosting) al algoritmo de "predecir los residuos" (Boosting Clásico), entendido por qué se llama "**Gradient**" Boosting (es un descenso de gradiente sobre una función de pérdida) y finalmente visto cómo **XGBoost/LightGBM** lo perfeccionan usando derivadas de segundo orden (el Hessiano).

Ahora, ¡vamos al laboratorio a aplicar esto!